### 문제 1 llm basic invoke

In [ ]:
from langchain_openai import ChatOpenAI
from langchain.document_loaders import TextLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser

# step 1
llm = ChatOpenAI(model="gpt-4o-mini", temperature=0)

# step 2
prompt = PromptTemplate.from_template(
    """
    You must answer the following question in one word.
    #Question:
    {question}
    """
    )

parser = StrOutputParser()

# step 5
# chain의 순서는 의미가 있구나
# | 연산자는 순차적으로 연산 결과를 다음 인자에 전달하는 것이기 떄문이다..
chain = prompt | llm | parser

question = "Where is the capital of South Korea?"
chain.invoke(question)


'Seoul'

### 문제 2 few shot prompt

In [18]:
from langchain_core.prompts import FewShotPromptTemplate

examples = [
    {
        "question": "스티브 잡스와 아인슈타인 중 누가 더 오래 살았나요?",
        "answer": """이 질문에 추가 질문이 필요한가요: 예.
추가 질문: 스티브 잡스는 몇 살에 사망했나요?
중간 답변: 스티브 잡스는 56세에 사망했습니다.
추가 질문: 아인슈타인은 몇 살에 사망했나요?
중간 답변: 아인슈타인은 76세에 사망했습니다.
최종 답변은: 아인슈타인
""",
    },
    {
        "question": "네이버의 창립자는 언제 태어났나요?",
        "answer": """이 질문에 추가 질문이 필요한가요: 예.
추가 질문: 네이버의 창립자는 누구인가요?
중간 답변: 네이버는 이해진에 의해 창립되었습니다.
추가 질문: 이해진은 언제 태어났나요?
중간 답변: 이해진은 1967년 6월 22일에 태어났습니다.
최종 답변은: 1967년 6월 22일
""",
    },
    {
        "question": "율곡 이이의 어머니가 태어난 해의 통치하던 왕은 누구인가요?",
        "answer": """이 질문에 추가 질문이 필요한가요: 예.
추가 질문: 율곡 이이의 어머니는 누구인가요?
중간 답변: 율곡 이이의 어머니는 신사임당입니다.
추가 질문: 신사임당은 언제 태어났나요?
중간 답변: 신사임당은 1504년에 태어났습니다.
추가 질문: 1504년에 조선을 통치한 왕은 누구인가요?
중간 답변: 1504년에 조선을 통치한 왕은 연산군입니다.
최종 답변은: 연산군
""",
    },
    {
        "question": "올드보이와 기생충의 감독이 같은 나라 출신인가요?",
        "answer": """이 질문에 추가 질문이 필요한가요: 예.
추가 질문: 올드보이의 감독은 누구인가요?
중간 답변: 올드보이의 감독은 박찬욱입니다.
추가 질문: 박찬욱은 어느 나라 출신인가요?
중간 답변: 박찬욱은 대한민국 출신입니다.
추가 질문: 기생충의 감독은 누구인가요?
중간 답변: 기생충의 감독은 봉준호입니다.
추가 질문: 봉준호는 어느 나라 출신인가요?
중간 답변: 봉준호는 대한민국 출신입니다.
최종 답변은: 예
""",
    },
]
example_prompt = PromptTemplate.from_template(
    "question:\n{question}, Answer:\n {answer}"
)
prompt = FewShotPromptTemplate(
    examples=examples,
    example_prompt=example_prompt,
    suffix="question:\n{question}, Answer:\n",
    input_variables=["question"]
)
chain = prompt | llm | parser
llm = ChatOpenAI(model="gpt-4o-mini")
parser = StrOutputParser()

question = "Google이 창립된 연도에 Bill Gates의 나이는 몇 살인가요?"

print(f"{chain.invoke({'question': question})}")

이 질문에 추가 질문이 필요한가요: 예.  
추가 질문: Google은 몇 년에 창립되었나요?  
중간 답변: Google은 1998년에 창립되었습니다.  
추가 질문: Bill Gates는 몇 년에 태어났나요?  
중간 답변: Bill Gates는 1955년에 태어났습니다.  
추가 질문: 1998년에는 Bill Gates가 몇 살이었나요?  
중간 답변: 1998년에는 Bill Gates가 43세였습니다.  
최종 답변은: 43세


### 3. basic rag

In [37]:

from langchain.embeddings import OpenAIEmbeddings
from langchain.vectorstores import Chroma
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnablePassthrough, RunnableMap

loader = TextLoader(file_path, encoding="utf-8")
text_splitter = RecursiveCharacterTextSplitter(chunk_size=300, chunk_overlap=50)
documents = loader.load_and_split(text_splitter=text_splitter)
embeddings = OpenAIEmbeddings()
vectorstore = Chroma(embedding_function=embeddings)

retriever = vectorstore.as_retriever()

llm = ChatOpenAI(model="gpt-4o-mini", temperature=0)
prompt = ChatPromptTemplate(
    [
        ("system", "You are an assistant. Please answer the question following context. \n#Context:\n{context}"),
        ("user", "{question}")
    ]
)
parser = StrOutputParser()

# chain
chain = (
    {"context": retriever, "question": RunnablePassthrough()}
    | prompt
    | llm
    | parser
)
question = "What is Word2vec?"
chain.invoke(question)

'Word2vec is a group of related models used in natural language processing (NLP) to produce word embeddings, which are dense vector representations of words. Developed by a team led by Tomas Mikolov at Google in 2013, Word2vec uses neural networks to learn the relationships between words based on their context in large text corpora.\n\nThere are two main architectures for training Word2vec:\n\n1. **Continuous Bag of Words (CBOW)**: This model predicts a target word based on its surrounding context words. It takes the context words as input and tries to predict the center word.\n\n2. **Skip-gram**: This model works in the opposite way; it uses a target word to predict the surrounding context words. It is particularly effective for capturing relationships between words, especially for infrequent words.\n\nWord2vec is widely used because it can capture semantic meanings and relationships between words, allowing for tasks such as word similarity, analogy reasoning, and improving the perfor

In [ ]:
def generate_questions_from_text(text):
    """
    Generate questions from the given text using OpenAI's GPT model.

    Args:
        text: The input text from which to generate questions.

    Returns:
        List of generated questions.
    """
    prompt = f"Based on the following text, generate several questions:\n\n{text}\n\nQuestions:"
    
    response = ChatOpenAI(model="gpt-4o-mini", temperature=0)
    
    questions = [choice['message']['content'] for choice in response['choices']]
    return questions

# 사용 예시
doc_content = "Your document content here."
questions = generate_questions_from_text(doc_content)
for question in questions:
    print(question)


SyntaxError: invalid syntax (1923412328.py, line 1)